In [ ]:
import os
import time
import re
import pyautogui
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Lista de meses y años a descargar
# Lista de periodos
MESES_ANIOS = [
"S_2015_12_2016_05"
"S_2016_06_2016_11"
"S_2016_12_2017_05"
"S_2017_06_2017_11"
"S_2017_12_2018_05"
"S_2018_06_2018_11"
"S_2018_12_2019_05"
"S_2019_06_2019_11"
"S_2019_12_2020_05"
"S_2020_06_2020_08"]
def cargar_coordenadas(periodo, carpeta=r"C:\Users\u_sociales\Downloads\coordenadas"):
    path = os.path.join(carpeta, f"coordenadas_{periodo}.txt")
    if not os.path.exists(path):
        print(f"⚠️ Archivo no encontrado: {path}")
        return []
    coordenadas = []
    with open(path, "r") as f:
        for linea in f:
            try:
                x, y = map(int, linea.strip().split(","))
                coordenadas.append((x, y))
            except:
                print(f"⚠️ Línea inválida: {linea.strip()}")
    
    return coordenadas    
for periodo in MESES_ANIOS:
    zoom_realizado = False
    coordenadas = cargar_coordenadas(periodo)
    if not coordenadas:
        continue

    # Definir carpetas de descarga y extracción
    download_folder = fr"C:\Users\Public\Downloads\{periodo}"
    extract_folder = fr"C:\Users\Public\Downloads\{periodo}_EXTRAIDO"
    os.makedirs(download_folder, exist_ok=True)
    os.makedirs(extract_folder, exist_ok=True)

    # Configurar opciones de Chrome
    chrome_options = Options()
    prefs = {
        "download.default_directory": download_folder,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "safebrowsing.disable_download_protection": True,
        "profile.default_content_setting_values.automatic_downloads": 1,
    }
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Usar WebDriver Manager para gestionar ChromeDriver automáticamente
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Maximizar ventana y acceder al sitio web
    driver.maximize_window()
    driver.get("https://minagri-geoespacial.users.earthengine.app/view/mosaicosv3")
    
    wait = WebDriverWait(driver, 20)

    # Acercar pantalla solo una vez con doble doble clic
    if not zoom_realizado:
        time.sleep(10)  # Esperar a que cargue completamente el mapa
        pyautogui.moveTo(951, 625, duration=0.5)
        pyautogui.doubleClick()
        time.sleep(5)
        pyautogui.moveTo(951, 625, duration=0.5)
        pyautogui.doubleClick()
        zoom_realizado = False
        time.sleep(10)
    
        # Check Cuadrantes    
        try:
            # Esperar a que el checkbox de "Cuadrantes" esté presente
            wait = WebDriverWait(driver, 15)
            
            # Buscar el checkbox que está asociado al label "Cuadrantes"
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='label' and text()='Cuadrantes']/preceding-sibling::span[contains(@class, 'jfk-checkbox')]")))
            
            # Hacer clic en el checkbox
            checkbox.click()
            print("Checkbox de 'Cuadrantes' seleccionado correctamente.")
    
        except Exception as e:
            print(f"Error al hacer clic en el checkbox: {e}")
    
         # Selección del periodo en el dropdown
        try:
            boton_seleccionar = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "jfk-select")))
            boton_seleccionar.click()
            wait.until(EC.text_to_be_present_in_element_attribute(
            (By.CLASS_NAME, "jfk-select"), "aria-expanded", "true"))
            opcion = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{periodo}')]")))
            opcion.click()
            print(f"📅 Seleccionado periodo: {periodo}")
        except Exception as e:
            print(f"❌ Error al seleccionar el periodo {periodo}: {e}")
            driver.quit()
            continue
    
        time.sleep(5)
    
        # Check Cuadrantes    
        try:
            # Esperar a que el checkbox de "Cuadrantes" esté presente
            wait = WebDriverWait(driver, 10)
            
            # Buscar el checkbox que está asociado al label "Cuadrantes"
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='label' and text()='Cuadrantes']/preceding-sibling::span[contains(@class, 'jfk-checkbox')]")))
            
            # Hacer clic en el checkbox
            checkbox.click()
            print("Checkbox de 'Cuadrantes' seleccionado correctamente.")
    
        except Exception as e:
            print(f"Error al hacer clic en el checkbox: {e}")
    
        # Check Cuadrantes    
        try:
            # Esperar a que el checkbox de "Cuadrantes" esté presente
            wait = WebDriverWait(driver, 10)
            
            # Buscar el checkbox que está asociado al label "Cuadrantes"
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='label' and text()='Cuadrantes']/preceding-sibling::span[contains(@class, 'jfk-checkbox')]")))
            
            # Hacer clic en el checkbox
            checkbox.click()
            print("Checkbox de 'Cuadrantes' seleccionado correctamente.")
    
        except Exception as e:
            print(f"Error al hacer clic en el checkbox: {e}")
    
        time.sleep(10)
    
        # Expresión regular para encontrar el enlace
        pattern = re.compile(fr"PLANET_IR:C\d{{4}}_{periodo}")
        
        for x, y in coordenadas:
            # Mueve el cursor y hace clic en las nuevas coordenadas
            pyautogui.moveTo(x, y, duration=0.5)
            pyautogui.click()
            
            # Espera para que la página cargue el nuevo enlace
            time.sleep(30)
    
            try:
                # Buscar todos los enlaces en la página
                links = driver.find_elements(By.TAG_NAME, "a")
                
                # Filtrar los enlaces que coincidan con el patrón
                for link in links:
                    if pattern.match(link.text):
                        ActionChains(driver).move_to_element(link).click().perform()
                        print(f"Se hizo clic en el enlace: {link.text} en las coordenadas ({x}, {y}).")
                        break  # Salir del bucle una vez encontrado el enlace correcto
                else:
                    print(f"No se encontró un enlace con el formato esperado en ({x}, {y}).")
    
            except Exception as e:
                print(f"Error al hacer clic en el enlace en ({x}, {y}):", e)
    
            # Espera para que la descarga inicie
            time.sleep(30)

    # Espera un momento antes de cerrar el navegador
    time.sleep(5)

    # Cerrar el navegador
    driver.quit()
    print("Navegador cerrado exitosamente.")